In [208]:
import pandas as pd

laliga = pd.read_csv('../scraping/tables/laliga/laliga_all_seasons.csv', index_col=0)
table = pd.read_csv('../scraping/tables/variable_data.csv')

In [209]:
# Set column names for `laliga` from the first row of `table` (excluding the first column)
laliga.columns = table.iloc[0, 1:]

# Get columns to keep based on the second row of `table`
columns_to_keep = table.T.loc[table.iloc[1] == '1'].iloc[:, 0]

# Remove duplicate columns and keep only the desired ones
laliga = laliga.loc[:, ~laliga.columns.duplicated() & laliga.columns.isin(columns_to_keep)]

# Reorder columns: move key identifiers to the front
columns_to_front = ['season_id', 'match_id', 'team_name', 'team_id']
laliga = laliga[columns_to_front + [col for col in laliga.columns if col not in columns_to_front]]

# Split the position and save the 1st one
laliga['Position'] = laliga['Position'].str.split(',').str[0].str.strip()
# Resulting DataFrame

import numpy as np
positions = laliga["Position"].unique()
unique_positions = np.unique(np.concatenate([pos.split(",") for pos in positions]))
unique_positions

off_pos = ['FW', 'LW', 'RW']
mid_pos = ['AM', 'CM', 'DM', 'LM', 'RM']
def_pos = ['CB', 'LB', 'RB', 'WB']
gk_pos = ['GK']

def define_group(position):
    if position in off_pos:
        return "off"
    if position in mid_pos:
        return "mid"
    if position in def_pos:
        return "def"
    else:
        return 'gk'

for player, stat in laliga.iterrows():
    group = define_group(laliga.loc[player,"Position"])
    laliga.loc[player, "Group"] = group

In [210]:
str(laliga.columns)

"Index(['season_id', 'match_id', 'team_name', 'team_id', 'Player Name',\n       'Shirt Number', 'Nationality', 'Position', 'Age at Season Start',\n       'Minutes Played', 'Goals Scored', 'Assists Provided',\n       'Penalty Kicks Made', 'Penalty Kicks Attempted', 'Total Shots',\n       'Shots on Target', 'Yellow Cards', 'Red Cards', 'Interceptions Made',\n       'Non-Penalty Expected Goals (npxG)', 'Shot-Creating Actions',\n       'Goal-Creating Actions', 'Progressive Passes', 'Take-Ons Attempted',\n       'Successful Take-Ons', 'Total Passes Completed',\n       'Total Passes Attempted', 'Expected Assists (xA)',\n       'Key Passes (Leading to Shots)', 'Passes into Penalty Area',\n       'Crosses into Penalty Area', 'Through Balls Completed',\n       'Passes Offside', 'Tackles Won', 'Tackles in Defensive Third',\n       'Tackles in Middle Third', 'Tackles in Attacking Third',\n       'Shots Blocked', 'Clearances Made', 'Errors Leading to Opponent Shots',\n       'Live-Ball Touches', '

In [211]:
laliga = laliga[['season_id', 'match_id', 'team_name', 'team_id', 'Player Name', 'Shirt Number', 'Group', 'Nationality', 'Position', 'Age at Season Start', 'Minutes Played', 'Goals Scored', 'Assists Provided', 'Key Passes (Leading to Shots)', 'Penalty Kicks Won', 'Penalty Kicks Conceded', 'Yellow Cards', 'Red Cards', 'Second Yellow Cards', 'Saves', 'Shots on Target', 'Successful Take-Ons', 'Carries into Penalty Area', 'Tackles Won', 'Clearances Made', 'Goals Against', 'Penalty Kicks Made', 'Penalty Kicks Attempted']]

In [212]:
laliga

,season_id,match_id,team_name,team_id,Player Name,Shirt Number,Group,Nationality,Position,Age at Season Start,...,Second Yellow Cards,Saves,Shots on Target,Successful Take-Ons,Carries into Penalty Area,Tackles Won,Clearances Made,Goals Against,Penalty Kicks Made,Penalty Kicks Attempted
0,1,1,Almería,H,Adri Embarba,10.0,off,es ESP,LW,31-096,...,0,NaN,0,2,0,2,0,NaN,0,0
1,1,1,Almería,H,Alejandro Pozo Pozo,17.0,def,es ESP,RB,24-170,...,0,NaN,0,0,0,0,4,NaN,0,0
2,1,1,Almería,H,Diego Mariño,1.0,gk,es ESP,GK,33-094,...,0,2.0,0,0,0,0,1,2.0,0,0
3,1,1,Almería,H,Dion Lopy,6.0,mid,sn SEN,CM,21-190,...,0,NaN,0,0,0,0,0,NaN,0,0
4,1,1,Almería,H,Gonzalo Melero,11.0,mid,es ESP,CM,29-221,...,0,NaN,0,1,0,0,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79170,7,380,Real Sociedad,A,Raúl Navas,22.0,def,es ESP,CB,30-009,...,0,NaN,0,0,0,1,2,NaN,0,0
79171,7,380,Real Sociedad,A,Sergio Canales,16.0,off,es ESP,RW,27-093,...,0,NaN,0,0,0,0,0,NaN,0,0
79172,7,380,Real Sociedad,A,Willian José,12.0,off,br BRA,FW,26-178,...,0,NaN,1,2,0,0,0,NaN,0,0
79173,7,380,Real Sociedad,A,Xabi Prieto,10.0,mid,es ESP,DM,34-264,...,0,NaN,0,0,0,0,0,NaN,0,0


In [213]:
import pandas as pd

# Step 1: Ensure 'Goals Against' is numeric
laliga['Goals Against'] = pd.to_numeric(laliga['Goals Against'], errors='coerce')

# Step 2: Extract "Goals Against" from GK only (group by season, match, and team)
ga_mapping = (
    laliga[laliga['Group'] == 'gk']
    .groupby(['season_id', 'match_id', 'team_id'], as_index=False)['Goals Against']
    .max()  # Assuming all GKs have the same value, but we take the max just in case
)

# Step 3: Merge to add "Goals Against" to all players (except GK)
laliga = laliga.merge(ga_mapping, on=['season_id', 'match_id', 'team_id'], how='left', suffixes=('', '_gk'))

# Step 4: Correctly update "Goals Against" **only for non-GK players**
laliga.loc[laliga['Group'] != 'gk', 'Goals Against'] = laliga['Goals Against_gk']

# Step 5: Drop the extra column
laliga.drop(columns=['Goals Against_gk'], inplace=True)

In [214]:
laliga.columns

Index(['season_id', 'match_id', 'team_name', 'team_id', 'Player Name',
       'Shirt Number', 'Group', 'Nationality', 'Position',
       'Age at Season Start', 'Minutes Played', 'Goals Scored',
       'Assists Provided', 'Key Passes (Leading to Shots)',
       'Penalty Kicks Won', 'Penalty Kicks Conceded', 'Yellow Cards',
       'Red Cards', 'Second Yellow Cards', 'Saves', 'Shots on Target',
       'Successful Take-Ons', 'Carries into Penalty Area', 'Tackles Won',
       'Clearances Made', 'Goals Against', 'Penalty Kicks Made',
       'Penalty Kicks Attempted'],
      dtype='object')

In [ ]:
def puntos(row):

    points = 0

    position = row['Group'] # off, mid, def, gk

    if row['Minutes Played'] > 60:
        points += 2
    elif (row['Minutes Played'] < 60) and (row['Minutes Played'] > 0):
        points += 1

    points += row['Goals Scored'] * (
    6 if position in ('def', 'gk') else 
    4 if position == 'off' else 
    5 if position == 'mid' else 0
    )

    points += row['Assists Provided'] * 3

    points += row['Key Passes (Leading to Shots)']

    if row['Goals Against'] == 0:
        points += (
            4 if position == 'gk' else
            3 if position == 'def' else 
            2 if position == 'mid' else 
            1 if position == 'off' else 0
        )

    points -= ( row['Penalty Kicks Attempted'] - row['Penalty Kicks Made'] ) * 2

    points += row['Penalty Kicks Won'] * 2

    points -= row['Penalty Kicks Conceded'] * 2

    if row['Goals Against'] > 0:
        if position in ('gk', 'def'):
            points -= (row['Goals Against'] // 2) * 2
        elif position in ('mid', 'off'):
            points -= (row['Goals Against'] // 2)
    
    points -= row['Yellow Cards'] * 1

    points -= row['Red Cards'] * 3

    points -= row['Second Yellow Cards'] * 1

    if position == 'gk':
        points += ( row['Saves'] // 2 )

    points += row['Shots on Target'] // 2

    points += row['Successful Take-Ons'] // 2

    points += row['Carries into Penalty Area'] // 2

    points += row['Clearances Made'] // 3

    points += row['Tackles Won'] // 5

    return points

In [245]:
laliga['puntos'] = laliga.apply(puntos, axis=1)

In [249]:
laliga.groupby('Player Name')['puntos'].sum().sort_values(ascending=False).head(20)

Player Name
Lionel Messi             1814.0
Iago Aspas               1634.0
Karim Benzema            1551.0
Antoine Griezmann        1485.0
Iñaki Williams           1326.0
Gerard Moreno            1286.0
Vinicius Júnior          1263.0
Daniel Parejo            1247.0
Mikel Oyarzabal          1205.0
Ángel Correa             1145.0
José Luis Morales        1141.0
Toni Kroos               1138.0
Luka Modrić              1123.0
Jan Oblak                1090.0
Luis Suárez              1082.0
Sergio Canales           1069.0
Marc-André ter Stegen    1020.0
Koke                     1016.0
Iker Muniain              974.0
Raúl García               962.0
Name: puntos, dtype: float64